# Fine-Tuning Language Models for Item Generation

### Learning Objectives

By the end of this notebook, you will:
1. Understand what fine-tuning is and when to use it
2. Know the difference between open-weight and closed models
3. Be able to prepare data for educational item generation
4. Run a complete fine-tuning workflow with LoRA
5. Evaluate model outputs and understand their limitations
6. Scale this approach to your own data

**Prerequisites:** Basic Python knowledge. No ML expertise required.

## Section 1: Conceptual Foundation

### 1.1 What is Fine-Tuning?

Fine-tuning refers to the process of taking a model that has been pre-trained on a large and diverse dataset, and then continuing its training on a smaller dataset that is specific to a target task or domain, typically with a lower learning rate and fewer updates, in order to adapt the representations learned by the model to the new task while retaining generalization ability (Howard & Ruder, 2018; Han et al., 2021).

**Analogy:**
- Base model = Chef who knows thousands of recipes
- Fine-tuning = Teaching that chef your family's specific recipe format

**What Fine-Tuning DOES:**
- Teaches output format (e.g., Stem, options A-D, Answer, Rationale)
- Adapts writing style and tone
- Learns task-specific patterns
- Improves consistency

**What Fine-Tuning DOESN'T Do:**
- Add new factual knowledge
- Fix fundamental reasoning limitations
- Guarantee 100% accuracy

### 1.2 Levels of Model Adaptation

There are multiple ways to adapt a model to your task. Understanding these helps you choose the right approach:

| **Method** | **Data Needed** | **Best Use Case** |
|-------------|-----------------|-------------------|
| **Instruction Tuning** | ~10k–100k+ | General instruction-following ability (e.g., making a base LM act like ChatGPT) |
| **Fine-Tuning (LoRA / PEFT)** | ~50–1000+ | Style, tone, or domain consistency with limited data |
| **Full Fine-Tuning** | ~1000–50k+ | Deep customization for a narrow domain or specific behavior |
| **Domain-Adaptive Pretraining (Continued Pretraining)** | ~100k–millions | Adapting to a specialized field (e.g., medical, legal, finance) |
| **Pretraining from Scratch** | ~Millions–billions | Building a model for an entirely new language or modality |


**This notebook focuses on LoRA fine-tuning**

### 1.3 Why LoRA (Low-Rank Adaptation)?  

Traditional fine-tuning involves updating all parameters of a pre-trained model. While effective, this approach is computationally intensive, requires substantial memory resources, and can be very costly. By contrast, Low-Rank Adaptation (LoRA) (Hu et al., 2021) introduces small trainable low-rank matrices into specific layers (typically attention projections) while freezing the original model weights. This allows the model to adapt to new tasks without modifying the entire parameter set.

Conceptually, LoRA is similar to inserting sticky notes into a book, rather than rewriting the entire book.  

**Benefits of LoRA include:**  
- **Efficiency:** Training can be completed in minutes rather than hours  
- **Resource savings:** Requires significantly less GPU memory (≈8 GB) compared to full fine-tuning (>40 GB)  
- **Lower cost:** Fine-tuning can be achieved at a fraction of the expense (often a few dollars versus hundreds)  
- **Comparable performance:** In many cases, LoRA achieves accuracy close to full fine-tuning while maintaining efficiency  


### **Technical Detail:**
LoRA adds small trainable matrices (adapters) to attention layers. These adapters (only 0.1-1% of model size) capture your task-specific patterns while keeping the original model frozen.

![LoRA Diagram](lora_transformer_block.png)

### **i. Input Tokens / Hidden States**
This is where everything begins. The text you feed into the model is first broken down into tokens. Each token is represented by a number so the model can process it. If this layer is deeper in the network, these inputs are called hidden states, which are internal representations of meaning that flow from one layer to the next.


### **ii. Embedding Layer (frozen)**
Next, the tokens are passed through the **embedding layer**, which converts each token into a list of numbers capturing its meaning in a mathematical space (a vector). For example, “king” and “queen” end up with similar embeddings because they share related meanings. In LoRA fine-tuning, this layer is frozen. Freezing it ensures that the base model keeps its original understanding of language, rather than overwriting it during fine-tuning.

### **iii. Multi-Head Attention (frozen)**
The transformed vectors then go into the multi-head attention mechanism. “Attention” means the model learns which words in a sentence should look at or relate to which others. For instance, *“dog”* might attend strongly to *“barked.”* “Multi-head” means the model does this in several different ways at once, each head focusing on different relationships. During LoRA training, these attention weights are also not updated.

### **iv. LoRA Adapters (Q/K/V/O projections) (trainable)**
Instead of retraining the full attention system, LoRA adds small, efficient adapters to it. These adapters are inserted into the **Q, K, V, and O projections**. These are shorthand for the mathematical transformations that attention uses to compute relationships between tokens (“Query,” “Key,” “Value,” and “Output”). The adapters are trainable, meaning only these small modules learn from your new data.  
They slightly adjust the frozen attention mechanism by adding a learned correction, represented by a low-rank matrix **AB**. This approach drastically reduces the number of parameters that need to be updated.


### **v. Combined Output (z′ = x(W + AB))**
Here, the model combines its frozen weights (**W**) with the new LoRA updates (**AB**). The formula simply means: use the old knowledge from pretraining, plus a lightweight learned tweak from LoRA. The result is an updated hidden representation (**z′**) that reflects both what the model originally knew and what it’s just learned for your new task.


### **vi. Feed-Forward Network (frozen)**
After attention, the model passes these representations into a **feed-forward network (FFN)**. This part helps the model interpret the attention output and refine it into a deeper understanding of each token’s meaning. In LoRA fine-tuning, this layer also remains frozen, i.e, keeping the core language understanding stable.


### **vii. LoRA Adapters (optional in FFN) (trainable)**
Some implementations also insert LoRA adapters inside the **feed-forward network** itself. This gives the fine-tuning process a bit more flexibility, allowing the adapters to influence not only how tokens attend to each other but also how they’re transformed afterward. These adapters are again small and efficient, learning only the specific tweaks needed for the new task.

### **viii. Combined Output (h′ = h(W + AB))**
Just like before, the outputs from the frozen FFN and the trainable LoRA adapters are merged. This combined output (**h′**) now carries the model’s base knowledge plus the new insights the adapters have learned.  

### **ix. Output Hidden States**
Finally, the model produces **output hidden states**, a refined representations that summarize everything the model has learned at this stage. These hidden states can then move on to the next layer of the transformer or directly be used to predict the next word, classify text, or perform other tasks. At this point, your model behaves slightly differently, but it still retains all its original general knowledge.



### **A Note on QLoRA**

A more recent development is Quantized LoRA (QLoRA) (Dettmers et al., 2023). QLoRA builds on LoRA by combining it with quantization, storing the frozen model weights in lower precision (e.g., 4-bit instead of 16-bit). This further reduces GPU memory requirements, making it possible to fine-tune very large models (up to 65B parameters) on a single modern GPU.

In practice:

- LoRA alone reduces the number of trainable parameters.

- QLoRA adds memory compression via quantization, allowing even larger models to be fine-tuned efficiently.

This means researchers can experiment with models previously considered “too big” for their hardware, without compromising much on performance.

### 1.4 Key Hyperparameters

**r (rank):** Size of adapter matrices
- Higher = more capacity to learn patterns, but slower training and risk of overfitting.
- Typical range: 4-32. powers of 2 (like 4, 8, 16, 32) are hardware friendly.
- Our choice: **16**

**alpha (scaling):** How much to trust the adapters vs original weights
- Usually 2× the rank
- If alpha is too small, adapters may under-contribute. If it is too large, adapters may overwhelm frozen weights.
- Our choice: **32**

**Learning rate:** How fast the model learns
- Too high = unstable/diverges, too low = slow convergence
- Typical range: 1e-4 to 5e-4
- Our choice: **2e-4**

**Number of epochs:** How many times to see the entire dataset
- Too few = underfit (poor performance), too many = overfit (memorization)
- The “right” number depends on dataset size and noise. Small datasets (like 50–500 items) often need fewer epochs (2–5). Larger datasets (10k+) might need 10–20.
- Watch validation loss to find the sweet spot
- Our choice: **3** (it is a good starting point)

## Section 2: Model Selection Guide

### 2.1 Open-Weight vs Closed Models

#### Open-Weight Models (You Download and Run)

**Definition:** Models whose parameters have been publicly released. You download the actual model files and run them yourself.

**Advantages:**
- **Full control:** Run locally, no internet dependency
- **Data privacy:** Your data never leaves your environment
- **Cost predictability:** No per-token fees, only compute costs
- **Customization:** Can fine-tune, quantize, modify architecture
- **No rate limits:** Generate as much as your hardware allows
- **Transparency:** You can inspect model behavior and biases

**Challenges:**
- **Infrastructure needed:** You must manage hardware, GPUs, deployment
- **Technical expertise:** Need to understand model loading, inference optimization
- **Maintenance:** You handle updates, bug fixes, security
- **Quality varies:** Smaller models may not match GPT-4 quality

**Popular Open-Weight Models:**

| Model | Size | Best For | Hardware Needed |
|-------|------|----------|----------------|
| **TinyLlama-1.1B** | 1.1B | Fast prototyping, simple tasks | 4GB GPU / 8GB CPU |
| **Phi-3-mini** | 3.8B | High quality for size, STEM | 8GB GPU / 16GB CPU |
| **Mistral-7B** | 7B | General purpose, good quality | 16GB GPU / 32GB CPU |
| **Llama-3-8B** | 8B | Strong reasoning, instruction following | 16GB GPU / 32GB CPU |
| **Qwen-2.5-7B** | 7B | Multilingual, code generation | 16GB GPU / 32GB CPU |
| **Gemma-7B** | 7B | Google's model, well-behaved | 16GB GPU / 32GB CPU |
| **Llama-3-70B** | 70B | Near GPT-4 quality | 80GB GPU (A100) |

For this workshop, we will use TinyLlama-1.1B (fast training, works on free Colab)

#### Closed Commercial Models (API Access Only)

**Definition:** Models developed by companies but not released as downloadable weights. You interact through an API.

**Advantages:**
- **Fastest to start:** No setup needed, just API key
- **Managed infrastructure:** No hardware to maintain
- **Best quality:** GPT-4, Claude top the benchmarks
- **Always updated:** Automatic improvements
- **Strong tooling:** Strong SDKs (software development kits), integrations, and documentation
- **Reliability:** Enterprise-grade uptime guarantees

**Challenges:**
- **Data privacy:** Your data passes through their servers
- **Usage costs:** Pay per token (can add up)
- **Rate limits:** Requests per minute caps
- **Dependency:** Service outages affect you
- **Less customization:** Fine-tuning options exist but are constrained and costly
- **Black box:** Model weights and architecture details are undisclosed

**Popular Closed Models:**

| Model | Provider | Best For | Cost (per 1M tokens) |
|-------|----------|----------|---------------------|
| **GPT-4o** | OpenAI | General intelligence, complex reasoning | $2.50 - $10 |
| **Claude 3.5 Sonnet** | Anthropic | Writing, analysis, code | $3 - $15 |
| **Gemini Pro** | Google | Multimodal, long context | $0.50 - $7 |
| **GPT-3.5 Turbo** | OpenAI | Fast, affordable, decent quality | $0.50 - $1.50 |
| **Command R+** | Cohere | RAG, enterprise search | $3 - $15 |

**Note on Fine-Tuning Closed Models:**
- OpenAI and some others offer fine-tuning APIs
- Costs: $8-30 per million training tokens + inference costs
- Your fine-tuned model stays on their infrastructure
- Good option if you need GPT-4 quality with your specific format

### 2.2 Decision Framework

**Choose Open-Weight Models if:**
- Data privacy is critical (FERPA, proprietary test items)
- You need full control and customization
- You have technical team/infrastructure
- You'll generate high volume (cost efficiency)
- You want to fine-tune and iterate quickly

**Choose Closed Models if:**
- You want fastest time to value
- You need best possible quality
- You prefer managed service
- Volume is low to medium
- Data privacy is not a blocker


**For this workshop:** We use **TinyLlama (open-weight)** because:
- It is free to use
- We can train quickly on Colab
- Demonstrates the complete workflow
- Same techniques apply to larger open-weight models

Fine-tuning doesn’t always need heavy coding. Platforms like Hugging Face AutoTrain, Predibase, Google Vertex AI, AWS SageMaker, and OpenAI’s Fine-Tuning API let you adapt models with little to no code.

## Section 3: Hardware Requirements

### 3.1 Understanding Compute Needs

**The Three Key Resources:**

1. **VRAM (GPU Memory):** Holds model weights and activations during training
2. **RAM (System Memory):** Loads data, manages operations
3. **Compute (FLOPS - floating point operations per second):** Performs calculations (matrix multiplications)

**Memory Requirements for Different Models:**

| Model Size | Loading (inference) | Training (Full) | Training (LoRA) |
|------------|---------------------|-----------------|----------------|
| **1B params** | 2GB | 8GB | 4-6GB |
| **3B params** | 6GB | 24GB | 10-14GB |
| **7B params** | 14GB | 56GB | 20-28GB |
| **13B params** | 26GB | 104GB | 40-50GB |
| **70B params** | 140GB | N/A (impractical) | 150-200GB |

*Note: Numbers are approximate for 16-bit precision. 4-bit quantization reduces by ~75%*

### 3.2 GPU Training

**Why GPUs instead of CPUs (Your local machine)?**
- Designed for parallel matrix operations (exactly what neural networks need)
- 10-100× faster than CPU for deep learning
- Modern frameworks optimized for GPU

**GPU Options:**

| GPU | VRAM | Use Case | Access |
|-----|------|----------|--------|
| **T4** | 16GB | Learning, small models | Free (Colab, Kaggle) |
| **RTX 3060** | 12GB | Personal workstation | 300-400 dollars|
| **RTX 4090** | 24GB | Professional workstation | 1600 dollars|
| **A10** | 24GB | Cloud training | 1-2 dollar/hour |
| **A100 40GB** | 40GB | Large models | 2-4 dollar/hour |
| **A100 80GB** | 80GB | Very large models | 4-8 dollar/hour |

**For This Workshop:**
- **Google Colab (Free):** T4 GPU, 16GB VRAM
- **Training time:** ~8-12 minutes


### 3.3 Cloud Options Comparison

| Platform | GPU | Cost | Pros | Cons |
|----------|-----|------|------|------|
| **Google Colab (Free)** | T4 | 0 | No setup, easy sharing | 12hr limit, can disconnect |
| **Google Colab Pro** | T4/V100 | 10/month | Longer sessions, better GPUs | Still has limits |
| **Kaggle** | P100/T4 | 0 | 30hr/week GPU quota | Less flexible than Colab |
| **AWS SageMaker** | Various | Pay-per-use | Professional, scalable | Complex setup |
| **Lambda Labs** | A100 | 1.10/hr | Cheapest A100 access | Limited regions |
| **RunPod** | Various | 0.20-2/hr | Flexible, good value | Smaller community |


**Memory-Saving Tips:**
- Use gradient checkpointing: `gradient_checkpointing_enable()`
- Reduce LoRA rank: `r=4` instead of `r=16`
- Lower batch size: `BATCH_SIZE=1`
- Shorter sequences: `MAX_SEQ_LENGTH=256`
- Enable gradient accumulation to simulate larger batches

### 3.4 Data preparation for item generation

A crucial step in fine-tuning is the design of your training data. The way you frame examples directly determines how the model will behave after training. For item generation, we adopt a supervised instruction–response format, where each record explicitly pairs:

- Input → a clear instruction that communicates the task and any relevant context (e.g., grade level, subject, or topic).

- Output → the exact target response, in the precise style and structure you want the model to reproduce.

We use the JSON Lines (JSONL) format for this purpose, where each line represents one training instance. For example:

```json
{"input": "You are a K-12 science item writer. Produce a question in this style … Topic: Earth’s layers.",
 "output": "Stem: Which layer of the Earth is liquid?\nA. Crust\nB. Mantle\nC. Outer core\nD. Inner core\nAnswer: C\nRationale: The outer core is molten iron and nickel, unlike the solid inner core."}
```

- The input explains the style rules (for example, always include a 'Stem:' line and options A–D) and includes a topic or tag such as grade level.
- The output is exactly how you want the model to respond after training.

### 3.5 Training, validation, and testing

Even though we are “just fine‑tuning,” we want to be sure the model generalizes and does not merely memorize our training items.

- **Training set**: data that updates the adapters (the learnable parameters).
- **Validation set**: small held‑out split to monitor whether learning actually improves. If validation performance gets worse while training performance improves, the model may be overfitting.
- **Test set**: a final held‑out split used only after training to estimate how the model performs on new, unseen items.

## Section 4: Environment Setup

To fine-tune an open-weight model, we rely on a few Python libraries:

- Transformers – for model loading and tokenization.

- PEFT – for parameter-efficient fine-tuning (LoRA/QLoRA).

- Accelerate – to manage GPUs and distributed training.

- Datasets – to handle your JSONL data easily.

In [ ]:
# Install Required Libraries

!pip install -q transformers peft accelerate datasets

In [ ]:
# Import Libraries
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Imports successful!")

In [ ]:
# Check Hardware

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{'='*70}")
print("HARDWARE CHECK")
print(f"{'='*70}")
print(f"Device: {device}")

if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.1f} GB")
    print("GPU detected! Training will be fast (~10 minutes)")
else:
    print("No GPU detected. Training will be slow (~60+ minutes)")
    print("In Google Colab: Runtime > Change runtime type > T4 GPU")

## Section 5: Configuration

These are good defaults and you can use them as-is for your first run.

In [ ]:
# Configuration

# Data
DATA_PATH = "openbookqa_itemgen_500.jsonl"
TRAIN_SPLIT = 0.9

# Model
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "./finetuned_model"

# Training
NUM_EPOCHS = 3 # The number of times to iterate over the training data
BATCH_SIZE = 4 # Batch size per device
GRADIENT_ACCUM = 2 # Number of steps to accumulate gradients before updating
LEARNING_RATE = 2e-4 # Learning rate for optimizer
MAX_SEQ_LENGTH = 512 # Maximum sequence length for inputs

# LoRA
LORA_R = 16 
LORA_ALPHA = 32 
LORA_DROPOUT = 0.05 

print(f"{'='*70}")
print("CONFIGURATION SUMMARY")
print(f"{'='*70}")
print(f"Model: {MODEL_NAME}")
print(f"Training samples: ~{int(500 * TRAIN_SPLIT)}") # We are using 500 observations in the data
print(f"Validation samples: ~{int(500 * (1-TRAIN_SPLIT))}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM}")
print(f"LoRA rank: {LORA_R}")
print(f"{'='*70}\n")

## Section 6: Data Inspection

Let's look at our training data before we start.

In [ ]:
# Inspect Data

print(f"{'='*70}")
print("DATA INSPECTION")
print(f"{'='*70}\n")

if not Path(DATA_PATH).exists():
    print(f"❌ Error: {DATA_PATH} not found!")
    print("\n To upload your file in Colab:")
    print("1. Click the folder icon on the left")
    print("2. Click the upload icon")
    print("3. Select your JSONL file")
    raise FileNotFoundError(f"{DATA_PATH} not found")

with open(DATA_PATH, 'r') as f:
    first_line = json.loads(f.readline())
    print("✅ File found and valid JSON!")
    print(f"\nFirst example:")
    print(f"-" * 70)
    print(f"INPUT (instruction):")
    print(f"{first_line['input'][:200]}...")
    print(f"\nOUTPUT (target format):")
    print(f"{first_line['output'][:200]}...")
    print(f"-" * 70)

## Section 7: Load Model and Tokenizer

In [ ]:
# Load Tokenizer

print(f"\n{'='*70}")
print("LOADING MODEL AND TOKENIZER")
print(f"{'='*70}\n")

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Tokenizer loaded")
print(f"   Vocabulary size: {len(tokenizer):,} tokens")

In [ ]:
# Load Base Model

print("\nLoading base model ...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="auto" if device == "cuda" else None,
)

if device == "cpu":
    model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print("✅ Model loaded successfully!")
print(f"   Total parameters: {total_params:,}")
print(f"   Model size: ~{total_params * 2 / 1e9:.2f} GB")

## Section 8: Configure LoRA

In [ ]:
# Configure LoRA Adapters

print("\n⚡ Configuring LoRA...")
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"✅ Trainable: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## Section 9: Prepare Training Data

In [ ]:
# Load and Split Dataset

print(f"\n{'='*70}")
print("PREPARING TRAINING DATA")
print(f"{'='*70}\n")

print(" Loading dataset...")
dataset = load_dataset("json", data_files=DATA_PATH, split="train")
print(f"✅ Loaded {len(dataset)} examples")

dataset = dataset.train_test_split(test_size=1-TRAIN_SPLIT, seed=42)
train_size = len(dataset["train"])
val_size = len(dataset["test"])

print("\n Data split:")
print(f"   Training: {train_size} examples ({TRAIN_SPLIT*100:.0f}%)")
print(f"   Validation: {val_size} examples ({(1-TRAIN_SPLIT)*100:.0f}%)")

In [ ]:
# Format and Tokenize

def format_prompt(example):
    return {
        "text": f"{example['input']}\n\n{example['output']}{tokenizer.eos_token}"
    }

print("\n Formatting examples...")
train_data = dataset["train"].map(format_prompt, remove_columns=["input", "output"])
test_data = dataset["test"].map(format_prompt, remove_columns=["input", "output"])

def tokenize(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
    )
    return result

print(" Tokenizing...")
train_data = train_data.map(tokenize, batched=True, remove_columns=["text"])
test_data = test_data.map(tokenize, batched=True, remove_columns=["text"])

example_tokens = train_data[0]["input_ids"]
print(f"\n Example tokenization:")
print(f"   Number of tokens: {len(example_tokens)}")
print(f"   First 10 tokens: {example_tokens[:10]}")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

print("\n✅ Data preparation complete!")

## Section 10: Training

In [ ]:
# Configure Training

print(f"\n{'='*70}")
print("CONFIGURING TRAINING")
print(f"{'='*70}\n")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16= False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
)

print("✅ Training configuration set!")
print("\nKey settings:")
print(f"   Total steps: ~{(train_size // (BATCH_SIZE * GRADIENT_ACCUM)) * NUM_EPOCHS}")
print(f"   Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM}")
print(f"   Learning rate: {LEARNING_RATE}")
print("   Estimated time (GPU): ~8-12 minutes")

In [ ]:
# Train the Model

print(f"\n{'='*70}")
print("TRAINING THE MODEL")
print(f"{'='*70}\n")

print(" Starting training...")
print(" Watch the train_loss and eval_loss")
print("   - Both should decrease")
print("   - If eval_loss increases: overfitting!\n")

trainer.train()

print("\n✅ Training complete!")

# Show summary
history = trainer.state.log_history
final_train_loss = [x for x in history if 'loss' in x][-1]['loss']
final_eval_loss = [x for x in history if 'eval_loss' in x][-1]['eval_loss']

print("\n Final metrics:")
print(f"   Training loss: {final_train_loss:.4f}")
print(f"   Validation loss: {final_eval_loss:.4f}")

if final_eval_loss < final_train_loss * 1.2:
    print("\n✅ Good! No significant overfitting detected")
else:
    print("\n⚠️  Validation loss higher - possible overfitting")

## Section 11: Save Model

In [ ]:
# Save the Fine-Tuned Model

print(f"\n{'='*70}")
print("SAVING MODEL")
print(f"{'='*70}\n")

print("Saving fine-tuned model...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved to: {OUTPUT_DIR}")
print("\n To use this model later:")
print("   from peft import AutoPeftModelForCausalLM")
print(f"   model = AutoPeftModelForCausalLM.from_pretrained('{OUTPUT_DIR}')")

## Section 12: Evaluation - Before vs After Fine-Tuning

Let's see what the model learned.

In [ ]:
# Load Base Model for Comparison

print(f"\n{'='*70}")
print("BEFORE vs AFTER COMPARISON")
print(f"{'='*70}\n")

test_prompt = """You are a K-12 science item writer. Produce an item using this exact house style:
- One line starting with 'Stem:'
- Four options labeled 'A.' 'B.' 'C.' and 'D.' (each on its own line)
- One line starting with 'Answer:' with a single letter A–D
- One line starting with 'Rationale:' (short and factual)
[TOPIC]=General science
Use concise, age-appropriate language.

"""

print(" Loading base model for comparison...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
)
if device == "cpu":
    base_model = base_model.to(device)
base_model.eval()
model.eval()

def generate_text(model, prompt, max_tokens=250):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

print("\n" + "─"*70)
print("BEFORE Fine-Tuning (Base Model)")
print("─"*70)
print("\nGenerating...")
before_output = generate_text(base_model, test_prompt)
print(before_output[:500])

print("\n" + "─"*70)
print("AFTER Fine-Tuning")
print("─"*70)
print("\nGenerating...")
after_output = generate_text(model, test_prompt)
print(after_output[:500])

In [ ]:
# Generate more Items

NUM_ITEMS = 5  # Change this number

topics = ['General science', 'Life science (plants)', 'Life science (animals)', 'Earth science', 'Space science', 'Physical science'] #Topics in the dataset

model.eval()

for i, topic in enumerate(topics[:NUM_ITEMS], 1):
    prompt = f"""You are a K-12 science item writer. Produce an item using this exact house style:
- One line starting with 'Stem:'
- Four options labeled 'A.' 'B.' 'C.' and 'D.' (each on its own line)
- One line starting with 'Answer:' with a single letter A–D
- One line starting with 'Rationale:' (short and factual)
[TOPIC]={topic}
Use concise, age-appropriate language.

"""

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7, do_sample=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):]

    print(f"\n{'='*70}")
    print(f"Item {i} - {topic}")
    print(f"{'='*70}")
    print(result[:500])  # First 500 chars

## Conclusion

Congratulations! You've successfully completed this fine-tuning workshop!

#### What You've Accomplished:
- ✅ Fine-tuned a language model with LoRA
- ✅ Understood key concepts and hyperparameters
- ✅ Learned about open-weight vs closed models
- ✅ Understood hardware requirements (GPU vs CPU)
- ✅ Compared base vs fine-tuned outputs


#### Key Takeaways:

1. **Fine-tuning teaches FORMAT, not FACTS**
   - The model already has knowledge
   - We taught it HOW to structure outputs

2. **LoRA is remarkably efficient**
   - Only 0.1-1% of parameters trained
   - 10-50× faster than full fine-tuning
   - Similar quality to full fine-tuning

3. **Data quality matters more than quantity**
   - 50 consistent examples > 500 inconsistent
   - Focus on representative examples
   - Always validate outputs

4. **Choose the right tool for the job**
   - Open-weight: Privacy, control, customization
   - Closed models: Quality, convenience, reliability
   - Hybrid approaches often work best

#### Further Reading:

**Research Papers:**
- LoRA: Low-Rank Adaptation: https://arxiv.org/abs/2106.09685
- QLoRA (4-bit training): https://arxiv.org/abs/2305.14314
- Instruction Tuning: https://arxiv.org/abs/2203.02155

**Documentation:**
- Hugging Face PEFT: https://huggingface.co/docs/peft
- Transformers Library: https://huggingface.co/docs/transformers
- Model Cards: https://huggingface.co/models

**Tutorials:**
- Fine-tuning guide: https://www.philschmid.de/instruction-tune-llama-2
- QLoRA tutorial: https://huggingface.co/blog/4bit-transformers-bitsandbytes
- Advanced techniques: https://github.com/huggingface/peft/tree/main/examples


#### Community & Support:

- **Hugging Face Forums:** https://discuss.huggingface.co/
- **Discord:** Join ML communities for live help
- **GitHub Issues:** Report bugs in libraries
- **Reddit:** r/LocalLLaMA, r/MachineLearning

#### Questions?
